In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [ ]:
df=pd.read_csv("./Datos/Index2018.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

* Tendencia
* Estacionaridad
* Ciclico
* Irregularidad

In [ ]:
df.spx.plot()

In [ ]:
df.spx.hist()

In [ ]:
import statsmodels.api as sm

In [ ]:
ciclo,tendencia=sm.tsa.filters.hpfilter(df["spx"])

In [ ]:
df["Tendencia_spx"]=tendencia

In [ ]:
df[["spx","Tendencia_spx"]][:20].plot()

In [ ]:
df["Medias_Moviles"]=df["spx"].rolling(5).mean()

In [ ]:
df[["spx","Tendencia_spx","Medias_Moviles"]][:20].plot()

In [ ]:
sm.tsa.seasonal_decompose(df[6050:]["spx"],model="additive",period=30).plot();

In [ ]:
df.tail(30)

In [ ]:
import scipy.stats 

In [ ]:
scipy.stats.probplot(df.spx[100:1000],plot=plt);

In [ ]:
df["date"]=pd.to_datetime(df["date"],format="%d/%m/%Y")

In [ ]:
df.info()

In [ ]:
# df["Mes"]=df["date"].dt.month
# df["Anyo"]=df["date"].dt.year

In [ ]:
df.set_index("date",inplace=True)

In [ ]:
df.head()

In [ ]:
df=df.asfreq('d')

In [ ]:
df.head()

In [ ]:
df.isna().sum()

* bfill
+ ffill

In [ ]:
df["spx"]=df["spx"].fillna(method="ffill")

In [ ]:
df.head()

In [ ]:
from sklearn.model_selection import train_test_split ## No sirve en series temporales

In [ ]:
tamanyo=int(len(df)*0.8)

In [ ]:
train=df.iloc[:tamanyo]
test=df.iloc[tamanyo:]

In [ ]:
train.tail()

In [ ]:
test.head()

## Estacionario

In [ ]:
import statsmodels.tsa.stattools as sts

$$H_0: \text{NO es estacioanrio}$$
$$H_1: \text{Es estacionario}$$

In [ ]:
sts.adfuller(df[:20].spx)

* Si el p-valor es superior o igual a 0,05 (por defecto) no se puede rechazar la hipotesis nula
* Si el p-valor es inferior a 0,05 (por defecto) se debe rechazar la hipotesis nula y por tanto aceptar la hipotesis alternativa


In [ ]:
import statsmodels.graphics.tsaplots as sgt

In [ ]:
#### ACF
sgt.plot_acf(df[:20].spx,lags=10,zero=False);

In [ ]:
sgt.plot_pacf(df[:20].spx,lags=10,zero=False);

## Paseo Aleatorio
Es la primera diferencia es estacionario e incorrelado

In [ ]:
paseo=pd.read_csv("./Datos/RandWalk.csv")

In [ ]:
paseo.head()

In [ ]:
paseo.date=pd.to_datetime(paseo.date,format="%d/%m/%Y")

In [ ]:
paseo.set_index(paseo.date,inplace=True)

In [ ]:
paseo.describe()

In [ ]:
paseo.price.plot()

In [ ]:
sts.adfuller(paseo.price)

In [ ]:
paseo["diferencia"]=paseo["price"]-paseo["price"].shift(1)
paseo["diferencia"]=paseo["diferencia"].fillna(method="bfill")
paseo["diferencia_log"]=np.log(paseo["price"]/paseo["price"].shift(1))
paseo["diferencia_log"]=paseo["diferencia_log"].fillna(method="bfill")

In [ ]:
sts.adfuller(paseo["diferencia"])

In [ ]:
sts.adfuller(paseo["diferencia_log"])

In [ ]:
paseo.head()

In [ ]:
sgt.plot_acf(paseo["diferencia"],lags=4,zero=False);

In [ ]:
sgt.plot_acf(paseo["diferencia_log"],lags=2,zero=False);

In [ ]:
train

In [ ]:
test.head()

In [ ]:
train["spx"].plot()

## Suavizado de los datos

### Tecnica de suavizado expoenecial EWM

$$ y_t= \cfrac{\sum\limits_{i=0}^t w_i x_{t_i}}{\sum\limits_{i=0}^t w_i}$$

In [ ]:
df["Suavizado_6"]=df["spx"].ewm(span=6).mean()

In [ ]:
df.head()

In [ ]:
df[["spx","Suavizado_6","Medias_Moviles"]].tail(30).plot()

#### Holt-Winters

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

In [ ]:
span=6
df["SES6"]=SimpleExpSmoothing(df["spx"]).fit(smoothing_level=1/span).fittedvalues.shift(-1)

In [ ]:
df.head()

In [ ]:
df[["spx","Suavizado_6","Medias_Moviles","SES6"]].tail(30).plot()

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
df["DES6"]=ExponentialSmoothing(df["spx"],trend="add").fit().fittedvalues.shift(-1)

In [ ]:
df[["spx","Suavizado_6","Medias_Moviles","SES6","DES6"]].tail(30).plot()

In [ ]:
df["DES6M"]=ExponentialSmoothing(df["spx"],trend="mul").fit().fittedvalues.shift(-1)

In [ ]:
df[["spx","Suavizado_6","Medias_Moviles","SES6","DES6","DES6M"]].tail(30).plot()

## Suavizado expoencial triple

In [ ]:
df["DES6T"]=ExponentialSmoothing(df["spx"],trend="add",seasonal="add",seasonal_periods=30).fit().fittedvalues.shift(-1)

In [ ]:
df[["spx","Suavizado_6","Medias_Moviles","SES6","DES6","DES6M","DES6T"]].tail(30).plot()

In [ ]:
modelo_Aditivo=ExponentialSmoothing(df["spx"],trend="add",seasonal="add",seasonal_periods=30).fit()
modelo_multipliativo=ExponentialSmoothing(df["spx"],trend="mul",seasonal="mul",seasonal_periods=30).fit()

In [ ]:
prediccion=modelo_Aditivo.forecast(36)
prediccion_M=modelo_multipliativo.forecast(36)

In [ ]:
df["spx"].tail(30).plot()
prediccion.plot(label="Prediccion")
prediccion_M.plot(label="Prediccion_M")
plt.legend();

In [ ]:
modelo_Aditivo=ExponentialSmoothing(train["spx"],trend="add",seasonal="add",seasonal_periods=30).fit()
modelo_multipliativo=ExponentialSmoothing(train["spx"],trend="mul",seasonal="mul",seasonal_periods=30).fit()

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
prediccion=modelo_Aditivo.forecast(36)
prediccion_M=modelo_multipliativo.forecast(36)

In [ ]:
test.shape

In [ ]:
prediccion.shape

In [ ]:
Prueba=test.head(36)

In [ ]:
prediccion

In [ ]:
mean_absolute_percentage_error(prediccion,Prueba["spx"])

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,median_absolute_error

In [ ]:
mean_squared_error(prediccion,Prueba["spx"])**(1/2)

In [ ]:
mean_absolute_error(prediccion,Prueba["spx"])

In [ ]:
median_absolute_error(prediccion,Prueba["spx"])

## Modelo AR

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
modelo_Ar=SARIMAX(df.spx,order=(1,0,0))

In [ ]:
resultado=modelo_Ar.fit()

In [ ]:
resultado.summary()

$y_t=y_{t-1} + \sigma^2 $

In [ ]:
modelo_Ar2=SARIMAX(df.spx,order=(2,0,0))

In [ ]:
resultado2=modelo_Ar2.fit()

In [ ]:
resultado2.summary()

In [ ]:
modelo_Ar23=SARIMAX(df.spx,order=(5,0,0))

In [ ]:
resultado23=modelo_Ar23.fit()

In [ ]:
resultado23.summary()